<a href="https://colab.research.google.com/github/Tommaso-R-Marena/cryptic-ip-binding-sites/blob/main/notebooks/06_Protein_Engineering_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Engineering an IP6-Dependent Fluorescent Protein

**Goal**: Design a superfolder GFP variant that requires IP6 for proper folding
and fluorescence, enabling experimental validation of buried IP cofactor
mechanisms.

## Computational Pipeline

1. **Structure Preparation**: Load sfGFP and ADAR2 IP6-binding pocket
2. **Pocket Design**: Graft ADAR2-like cavity into sfGFP core
3. **Sequence Optimization**: Introduce 4-6 Lys/Arg for IP6 coordination
4. **Structural Modeling**: AlphaFold2 prediction of engineered variant
5. **MD Simulations**: Validate stability ± IP6 with OpenMM
6. **QM/MM Calculations**: Quantum-mechanical energy barriers
7. **Experimental Design**: Protocols for wet-lab validation

## Design Criteria (from ADAR2)

- Pocket depth: >15 Å
- SASA: <5 Å²
- Electrostatic potential: >+5 kT/e
- Coordinating residues: 4-6 Lys/Arg
- Volume: 400-600 Å³ (IP6 size)

## 0. Setup and Dependencies

In [ ]:
import sys
import os
from pathlib import Path


IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print('=== Google Colab Environment ===')
    print('Installing computational biology stack...')
    
    # Core dependencies
    !pip install -q biopython requests pandas matplotlib seaborn \
                     numpy scipy
    
    # Molecular dynamics
    !pip install -q openmm mdtraj
    
    # Protein design & chemistry
    !pip install -q py3Dmol biotite prody rdkit
    
    print('\n✓ Dependencies installed')
    
    # Clone repository
    if not Path('cryptic-ip-binding-sites').exists():
        repo_url = 'https://github.com/Tommaso-R-Marena'
        repo_name = 'cryptic-ip-binding-sites'
        !git clone {repo_url}/{repo_name}.git
        os.chdir(repo_name)
    
    sys.path.insert(0, str(Path.cwd()))
    work_dir = Path('notebook_data/protein_engineering')
    work_dir.mkdir(parents=True, exist_ok=True)
    
    print('✓ Colab setup complete!')

else:
    sys.path.insert(0, str(Path.cwd().parent))
    work_dir = Path('notebook_data/protein_engineering')
    work_dir.mkdir(parents=True, exist_ok=True)
    print('✓ Local setup complete!')


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from Bio import PDB
from Bio.PDB import PDBIO, Select
from scipy.spatial.distance import cdist
from scipy import stats
import requests
import gzip
import json
import warnings

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
%matplotlib inline

print('✓ Core imports successful')
